In [3]:
! pip install datasets
from datasets import load_dataset
dataset = load_dataset("Skylion007/openwebtext", split="train")


/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/80 [00:00<?, ?it/s]

In [4]:
! pip install tiktoken
import tiktoken
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from tiktoken import get_encoding
import torch
import torch.nn as nn
from torch.nn import functional as F


In [4]:
# class TextDataset(torch.utils.data.Dataset):
#     def __init__(self, texts, tokenizer):
#         self.texts = texts
#         self.tokenizer = tokenizer
#         self.block_size = 32

#     def __len__(self):
#         return len(self.texts)

#     def __getitem__(self, idx):
#         data = " ".join(self.texts)
#         self.encoded_text = self.tokenizer.encode(data) # tokenizing in batches
#         x = self.encoded_text[idx:idx+self.block_size]
#         y = self.encoded_text[idx+1:idx+self.block_size+1]
#         return x, y


In [4]:
print(len(dataset))

8013769


In [5]:
# print(len(dataset))
# print(dataset[0])
# print(dataset[1])
# print(dataset)

# enc = tiktoken.get_encoding('gpt2')
# vocab_size = enc.n_vocab

# dataset_subset =  dataset.select(range(100000))
# tokenizer = get_encoding('gpt2')
# texts = [text['text'] for text in dataset_subset]

# # Initialize dataset
# text_dataset = TextDataset(texts, tokenizer)

# # Split dataset into training and validation sets
# train_size = int(0.9 * len(text_dataset))
# val_size = len(text_dataset) - train_size
# train_dataset, val_dataset = random_split(text_dataset, [train_size, val_size])

# # DataLoader
# batch_size = 16
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [6]:
enc = tiktoken.get_encoding('gpt2')
vocab_size = enc.n_vocab

# print(len(dataset))
# print(dataset[0])
# print(dataset[1])
# print(dataset)

dataset_subset =  dataset.select(range(2000000))
dataset_subset = dataset_subset['text']
n = int(0.8*len(dataset_subset)) # first 90% will be train, rest val
train_data = dataset_subset[:n]
val_data = dataset_subset[n:]

In [7]:



# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel? #16
block_size = 32 # what is the maximum context length for predictions? #32
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

enc = tiktoken.get_encoding('gpt2')

# data loading
def get_batch(split):
    data = train_data if split == 'train' else val_data
    # print(len(data))
    # print(batch_size*block_size)
    random_int = torch.randint(len(data)- batch_size*block_size, (1,))
    start_index = int(random_int.item())

    data = train_data if split == 'train' else val_data
    sampled_elements = data[start_index:start_index + 4]
    combined_text = " ".join(sampled_elements)

    data = torch.tensor(enc.encode(combined_text), dtype=torch.long)

    ix = torch.randint(len(data) - block_size, (batch_size,)) #generate random number in the shape of block size
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)

    # print("printing once")
    # print(x)
    # print(y)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(enc.decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

6.684497 M parameters
step 0: train loss 10.9993, val loss 11.0053
step 100: train loss 7.7129, val loss 7.7183
step 200: train loss 7.5128, val loss 7.5193
step 300: train loss 7.3907, val loss 7.3888
step 400: train loss 7.2525, val loss 7.2898
step 500: train loss 7.1715, val loss 7.1531
step 600: train loss 7.1752, val loss 7.1358
step 700: train loss 7.0072, val loss 7.0456
step 800: train loss 6.9900, val loss 6.9811
step 900: train loss 6.9380, val loss 6.8990
step 1000: train loss 6.8757, val loss 6.8539
step 1100: train loss 6.8264, val loss 6.8212
step 1200: train loss 6.8049, val loss 6.8203
step 1300: train loss 6.7650, val loss 6.7836
step 1400: train loss 6.7415, val loss 6.7563
step 1500: train loss 6.7103, val loss 6.6858
step 1600: train loss 6.6660, val loss 6.7098
step 1700: train loss 6.6251, val loss 6.6787
step 1800: train loss 6.6120, val loss 6.7045
step 1900: train loss 6.6481, val loss 6.6059
step 2000: train loss 6.5462, val loss 6.5889
step 2100: train loss 